### imports
***

In [1]:
%run preprocess.ipynb

/usr/local/lib/python3.7/dist-packages/numba/errors.py:137: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)
/home/iis519/.local/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)



(2079282, 8)
cpu: 48
used cpu: 16
77.18794895410538
(2079282, 217)
success
87.85991539557774


In [2]:
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
import multiprocessing as mp
import pandas as pd
import numpy as np
import re
import time
from tqdm.notebook import tqdm
from util import pre_process
import pickle
import gc
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')

In [3]:
data_path = '../../data_2020/'
model_path = 'models/'
other_path = 'others/'
n = 80
drop_no_content = True
drop_no_year = True
drop_keywords = False
drop_no_abstract = False
MIN_LEN = 7
workers = mp.cpu_count()//2
paper_thd = 47500
take = '_new'
num_leaves = 64
reg_alpha = 1
reg_lambda = 0.1
objective = 'binary'
max_depth = -1
learning_rate = 0.1
min_child_samples = 5
n_estimators = 5000
subsample = 0.8
colsample_bytree = 0.8
seed = 9487

### load data
***

In [4]:
# for prediction
test = pd.read_csv(data_path+'validation.csv')

# get meta
test_data = pd.read_csv(data_path+'test_data_merge_{}_{}.csv'.format(n, paper_thd))[['paper_id','description_id','journal','year','keywords']].fillna('')
abstract = pd.read_csv(data_path+'test_data_merge_{}_{}.csv'.format(n, paper_thd))[['abstract']].fillna('NO_CONTENT')
print(test_data.shape)

# drop
keep_idxs = []
if drop_no_content:
    test_have_content_idxs = test_data.index[test_data['journal'] != 'no-content'].tolist()
    keep_idxs += test_have_content_idxs
    print(len(keep_idxs))
if drop_no_year:
    test_have_year_idxs = test_data.index[test_data['year'] != ''].tolist()
    if len(keep_idxs) > 0:
        keep_idxs = list(set(test_have_year_idxs).intersection(set(keep_idxs)))
    else:
        keep_idxs += test_have_year_idxs
    print(len(keep_idxs))
if drop_keywords:
    test_no_keywords_idxs = test_data.index[test_data['keywords'] == ''].tolist()
    if len(keep_idxs) > 0:
        keep_idxs = list(set(test_no_keywords_idxs).intersection(set(keep_idxs)))
    else:
        keep_idxs += test_have_year_idxs
    print(len(keep_idxs))
if drop_no_abstract:
    test_have_abstract_idxs = abstract.index[abstract['abstract'] != 'NO_CONTENT'].tolist()
    if len(keep_idxs) > 0:
        keep_idxs = list(set(test_have_abstract_idxs).intersection(set(keep_idxs)))
    else:
        keep_idxs += test_have_abstract_idxs
    print(len(keep_idxs))
        
# sort indexs
keep_idxs = sorted(keep_idxs)
if not drop_no_content and not drop_no_year and not drop_keywords and not drop_no_abstract:
    keep_idxs = [i for i in range(test_data.shape[0])]
    
# get feature
test_x = pd.read_csv(data_path+'test_data_merge_{}_{}_featall.csv'.format(n, paper_thd))
print(test_x.shape)
test_x = test_x.iloc[keep_idxs]
test_data = test_data.iloc[keep_idxs]

test_x.shape, test_data.shape

(2079282, 5)
2079282
2079282
(2079282, 217)


((2079282, 217), (2079282, 5))

In [5]:
# # drop feature
# drop = 'psu_key'
# cols = [col for col in test_x.columns.values if not col.__contains__(drop)]
# train_x = train_x[cols]
# test_x = test_x[cols]
# train_x.shape, test_x.shape

### save & load model
***

In [6]:
# # save
# for i in range(len(models)):
#     with open(model_path+'lgb_{}'.format(i), 'wb') as f:
#         pickle.dump(models[i], f)

In [7]:
# load
models = []
for i in range(10):
    with open(model_path+'lgb{}_{}'.format(take, i), 'rb') as f:
        models.append(pickle.load(f))

### prediction
***

In [8]:
# # average
# test_pred = np.zeros(test_x.shape[0])
# for pred in test_preds:
#     test_pred += pred
# test_data['pred'] = test_pred/len(models)
# temp = test_data[['description_id','pred']].groupby('description_id')['pred'].agg(lambda x: list(x))
# test_data['preb_list'] = test_data['description_id'].map(temp)
# temp = test_data[['description_id','paper_id']].groupby('description_id')['paper_id'].agg(lambda x: list(x))
# test_data['paper_id_list'] = test_data['description_id'].map(temp)
# test_sub = test_data.drop_duplicates('description_id')

# def get_topn_paper_id(paper_id_list,preb_list,tag):
#     index = np.argsort(preb_list)
#     if len(index) < tag:
#         tag = len(index)
#         return np.nan
#     return paper_id_list[index[-tag]]

# test_sub['p1'] = list(map(lambda x,y:get_topn_paper_id(x,y,1),test_sub['paper_id_list'],test_sub['preb_list']))
# test_sub['p2'] = list(map(lambda x,y:get_topn_paper_id(x,y,2),test_sub['paper_id_list'],test_sub['preb_list']))
# test_sub['p3'] = list(map(lambda x,y:get_topn_paper_id(x,y,3),test_sub['paper_id_list'],test_sub['preb_list']))
# test_sub = test_sub[['description_id','p1','p2','p3']]

# sub = test[['description_id']]
# sub = sub.fillna('none')
# sub = sub.merge(test_sub, on='description_id', how='left')
# sub.loc[sub['description_id']=='none', 'description_id'] = ''
# sub.to_csv('predictions/prediction_{}.csv'.format(seed), index=False, sep=',', header=None)

In [9]:
%%time
# vote
def get_topn_paper_id(paper_id_list,preb_list,tag):
    index = np.argsort(preb_list)
    if len(index) < tag:
        tag = len(index)
        return np.nan
    return paper_id_list[index[-tag]]

did2vote = {did[0]: {} for did in test[['description_id']].fillna('none').values}
test_preds = [model.predict_proba(test_x)[:, 1] for model in tqdm(models)]

for test_pred in tqdm(test_preds):
    test_data['pred'] = test_pred
    temp = test_data[['description_id','pred']].groupby('description_id')['pred'].agg(lambda x: list(x))
    test_data['preb_list'] = test_data['description_id'].map(temp)
    temp = test_data[['description_id','paper_id']].groupby('description_id')['paper_id'].agg(lambda x: list(x))
    test_data['paper_id_list'] = test_data['description_id'].map(temp)
    test_sub = test_data.drop_duplicates('description_id')

    test_sub['p1'] = list(map(lambda x,y:get_topn_paper_id(x,y,1),test_sub['paper_id_list'],test_sub['preb_list']))
    test_sub['p2'] = list(map(lambda x,y:get_topn_paper_id(x,y,2),test_sub['paper_id_list'],test_sub['preb_list']))
    test_sub['p3'] = list(map(lambda x,y:get_topn_paper_id(x,y,3),test_sub['paper_id_list'],test_sub['preb_list']))
    test_sub['p4'] = list(map(lambda x,y:get_topn_paper_id(x,y,4),test_sub['paper_id_list'],test_sub['preb_list']))
    test_sub['p5'] = list(map(lambda x,y:get_topn_paper_id(x,y,5),test_sub['paper_id_list'],test_sub['preb_list']))
    test_sub['p6'] = list(map(lambda x,y:get_topn_paper_id(x,y,6),test_sub['paper_id_list'],test_sub['preb_list']))
    test_sub['p7'] = list(map(lambda x,y:get_topn_paper_id(x,y,7),test_sub['paper_id_list'],test_sub['preb_list']))
    test_sub['p8'] = list(map(lambda x,y:get_topn_paper_id(x,y,8),test_sub['paper_id_list'],test_sub['preb_list']))
    test_sub = test_sub[['description_id','p1','p2','p3','p4','p5','p6','p7','p8']]
    
    # vote
    weights = {1: 10, 2: 8, 3: 6, 4: 5, 5: 4, 6: 3, 7: 2, 8: 1}
    for row in test_sub.values:
        for i in range(1, 9):
            if row[i] not in did2vote[row[0]]:
                did2vote[row[0]][row[i]] = weights[i]
            else:
                did2vote[row[0]][row[i]] += weights[i]


CPU times: user 1h 3min 1s, sys: 50.4 s, total: 1h 3min 51s
Wall time: 4min 20s


In [10]:
test_sub = []
for did in tqdm(list(did2vote.keys())):
    tmp = [(key, did2vote[did][key]) for key in list(did2vote[did].keys())]
    tmp.sort(key=lambda x: x[1], reverse=True)
    top3 = []
    for pid in tmp:
        if type(pid[0]) != float:
            top3.append(pid[0])
            if len(top3) >= 3:
                break
    test_sub.append([did]+top3)
test_sub = pd.DataFrame(test_sub, columns=['description_id','p1','p2','p3'])

sub = test[['description_id']]
sub = sub.merge(test_sub, on='description_id', how='left')
sub.to_csv('predictions/prediction_{}_{}_{}.csv'.format(seed, paper_thd, n), index=False, sep=',', header=None)

In [11]:
sum(sum(test_preds)/len(test_preds[0]))/len(models)

0.006690240213476242

In [12]:
sub[pd.isnull(sub).any(axis=1)]

,description_id,p1,p2,p3
